In [ ]:
# INSTALL DEPENDENCIES
!pip install requests pandas -q

import requests
import time
import json
import csv
import pandas as pd
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from io import StringIO

print("="*60)
print("NSE OPTION CHAIN OI ANALYSIS")
print("="*60)

# Configuration
EQUITY_CSV_URL = "https://nsearchives.nseindia.com/content/equities/EQUITY_L.csv"
OPTION_CHAIN_API = "https://www.nseindia.com/api/option-chain-equities?symbol={}"
TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
JSON_OUTPUT = f"nse_option_chain_data_{TIMESTAMP}.json"
CSV_OUTPUT = f"nse_oi_analysis_{TIMESTAMP}.csv"

# NSE requires proper headers to avoid 403 errors
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.nseindia.com/option-chain",
    "Connection": "keep-alive"
}

# Create session with proper headers
session = requests.Session()
session.headers.update(HEADERS)

def initialize_session():
    """Initialize session by visiting NSE homepage to get cookies"""
    try:
        print("\n[1/5] Initializing NSE session...")
        response = session.get("https://www.nseindia.com", timeout=10)
        time.sleep(1)  # Small delay after initialization
        print("✓ Session initialized successfully")
        return True
    except Exception as e:
        print(f"✗ Session initialization failed: {e}")
        return False

def get_nse_symbols():
    """Fetch list of all NSE equity symbols from official CSV"""
    try:
        print("\n[2/5] Fetching NSE equity symbols list...")
        response = session.get(EQUITY_CSV_URL, timeout=15)
        response.raise_for_status()

        # Parse CSV
        content = response.text
        reader = csv.DictReader(StringIO(content))
        symbols = []

        for row in reader:
            symbol = row.get('SYMBOL') or row.get('Symbol') or row.get('symbol')
            if symbol and symbol.strip():
                symbols.append(symbol.strip())

        print(f"✓ Found {len(symbols)} NSE symbols")
        print(f"  Sample symbols: {symbols[:10]}")
        return symbols
    except Exception as e:
        print(f"✗ Failed to fetch symbols: {e}")
        return []

def fetch_option_chain_data(symbol):
    """Fetch option chain data for a single symbol"""
    try:
        url = OPTION_CHAIN_API.format(symbol)
        response = session.get(url, timeout=20)

        if response.status_code == 200:
            data = response.json()
            records = data.get("records", {}).get("data", [])

            # Aggregate OI data
            oi_current = 0
            oi_change = 0

            for record in records:
                # Process Call Options (CE)
                if "CE" in record:
                    ce_data = record["CE"]
                    oi_current += ce_data.get("openInterest", 0)
                    oi_change += ce_data.get("changeinOpenInterest", 0)

                # Process Put Options (PE)
                if "PE" in record:
                    pe_data = record["PE"]
                    oi_current += pe_data.get("openInterest", 0)
                    oi_change += pe_data.get("changeinOpenInterest", 0)

            # Calculate previous session OI
            oi_previous = oi_current - oi_change

            return {
                "symbol": symbol,
                "oi_current": int(oi_current),
                "oi_previous": int(oi_previous),
                "oi_change": int(oi_change),
                "status": "success"
            }
        else:
            return {
                "symbol": symbol,
                "error": f"HTTP {response.status_code}",
                "status": "failed"
            }
    except Exception as e:
        return {
            "symbol": symbol,
            "error": str(e),
            "status": "failed"
        }

def fetch_all_option_chains(symbols, max_symbols=100, max_workers=5):
    """Fetch option chain data for multiple symbols with rate limiting"""
    print(f"\n[3/5] Fetching option chain data...")
    print(f"  Processing up to {max_symbols} symbols with {max_workers} workers")
    print(f"  (Limited for demo - adjust max_symbols for full analysis)")

    # Limit symbols for faster execution (remove limit for production)
    symbols_to_process = symbols[:max_symbols]
    results = []
    successful = 0
    failed = 0

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        future_to_symbol = {
            executor.submit(fetch_option_chain_data, symbol): symbol
            for symbol in symbols_to_process
        }

        # Process completed tasks
        for i, future in enumerate(as_completed(future_to_symbol), 1):
            result = future.result()
            results.append(result)

            if result.get("status") == "success":
                successful += 1
            else:
                failed += 1

            # Progress indicator
            if i % 10 == 0:
                print(f"  Progress: {i}/{len(symbols_to_process)} | Success: {successful} | Failed: {failed}")

            # Rate limiting to avoid NSE blocks
            time.sleep(0.5)

    print(f"\n✓ Fetching complete: {successful} successful, {failed} failed")
    return results

def save_fetched_data(data):
    """Save fetched data to JSON and CSV files"""
    print(f"\n[4/5] Saving fetched data...")

    # Save as JSON
    output_data = {
        "fetched_at": datetime.now().isoformat(),
        "total_symbols": len(data),
        "data": data
    }

    with open(JSON_OUTPUT, 'w') as f:
        json.dump(output_data, f, indent=2)
    print(f"✓ Saved JSON: {JSON_OUTPUT}")

    # Also save as CSV for easy inspection
    successful_data = [d for d in data if d.get("status") == "success"]
    if successful_data:
        df = pd.DataFrame(successful_data)
        df.to_csv(CSV_OUTPUT.replace('analysis', 'raw_data'), index=False)
        print(f"✓ Saved CSV: {CSV_OUTPUT.replace('analysis', 'raw_data')}")

    return JSON_OUTPUT


def load_fetched_data(json_file):
    """Load previously fetched data from JSON file"""
    print(f"\n[5/5] Loading and analyzing data...")
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data.get("data", [])

def calculate_oi_percentage_change(data):
    """Calculate percentage change in OI for each stock"""
    analysis_results = []

    for record in data:
        if record.get("status") != "success":
            continue

        symbol = record["symbol"]
        oi_current = record.get("oi_current", 0)
        oi_previous = record.get("oi_previous", 0)
        oi_change = record.get("oi_change", 0)

        # Calculate percentage change
        # Only consider positive changes (OI buildup)
        if oi_previous > 0 and oi_change > 0:
            pct_change = (oi_change / oi_previous) * 100

            analysis_results.append({
                "symbol": symbol,
                "oi_current": oi_current,
                "oi_previous": oi_previous,
                "oi_change": oi_change,
                "pct_change": pct_change
            })

    return analysis_results

def rank_and_display_top_stocks(analysis_results, top_n=5):
    """Rank stocks by OI percentage increase and display top N"""
    # Sort by percentage change in descending order
    ranked = sorted(analysis_results, key=lambda x: x["pct_change"], reverse=True)

    # Get top N
    top_stocks = ranked[:top_n]

    # Display results
    print("\n" + "="*60)
    print(f"TOP {top_n} STOCKS WITH HIGHEST OI PERCENTAGE INCREASE")
    print("="*60)
    print(f"{'Rank':<6} {'Symbol':<15} {'% Change':<12} {'Current OI':<15} {'Previous OI':<15} {'Change':<15}")
    print("-"*60)

    for i, stock in enumerate(top_stocks, 1):
        print(f"{i:<6} {stock['symbol']:<15} {stock['pct_change']:>10.2f}% {stock['oi_current']:>14,} {stock['oi_previous']:>14,} {stock['oi_change']:>14,}")

    print("="*60)

    # Save analysis to CSV
    df = pd.DataFrame(top_stocks)
    df.to_csv(CSV_OUTPUT, index=False)
    print(f"\n✓ Analysis saved to: {CSV_OUTPUT}")

    return top_stocks

def main():
    """Main execution function"""
    start_time = time.time()

    # Part 1: Data Fetching
    if not initialize_session():
        print("\n✗ Failed to initialize session. Exiting.")
        return

    symbols = get_nse_symbols()
    if not symbols:
        print("\n✗ No symbols found. Exiting.")
        return

    fetched_data = fetch_all_option_chains(symbols, max_symbols=100, max_workers=5)
    json_file = save_fetched_data(fetched_data)

    # Part 2: Analysis & Reporting
    loaded_data = load_fetched_data(json_file)
    analysis_results = calculate_oi_percentage_change(loaded_data)

    if not analysis_results:
        print("\n✗ No valid data for analysis. Try increasing max_symbols or check NSE availability.")
        return

    top_stocks = rank_and_display_top_stocks(analysis_results, top_n=5)

    # Summary
    elapsed = time.time() - start_time
    print(f"\n✓ Analysis complete in {elapsed:.1f} seconds")
    print(f"✓ Analyzed {len(analysis_results)} stocks with option chain data")
    print(f"\nOutput files:")
    print(f"  - Raw data (JSON): {json_file}")
    print(f"  - Analysis (CSV): {CSV_OUTPUT}")

    # Display files for download
    from google.colab import files
    print(f"\n📥 Download files using: files.download('{CSV_OUTPUT}')")

# Run the analysis
if __name__ == "__main__":
    main()


NSE OPTION CHAIN OI ANALYSIS

[1/5] Initializing NSE session...
✓ Session initialized successfully

[2/5] Fetching NSE equity symbols list...
✓ Found 2209 NSE symbols
  Sample symbols: ['20MICRONS', '21STCENMGM', '360ONE', '3IINFOLTD', '3MINDIA', '3PLAND', '5PAISA', '63MOONS', 'A2ZINFRA', 'AAATECH']

[3/5] Fetching option chain data...
  Processing up to 100 symbols with 5 workers
  (Limited for demo - adjust max_symbols for full analysis)
  Progress: 10/100 | Success: 10 | Failed: 0
  Progress: 20/100 | Success: 20 | Failed: 0
  Progress: 30/100 | Success: 30 | Failed: 0
  Progress: 40/100 | Success: 40 | Failed: 0
  Progress: 50/100 | Success: 45 | Failed: 5
  Progress: 60/100 | Success: 47 | Failed: 13
  Progress: 70/100 | Success: 48 | Failed: 22
  Progress: 80/100 | Success: 48 | Failed: 32
  Progress: 90/100 | Success: 48 | Failed: 42
  Progress: 100/100 | Success: 48 | Failed: 52

✓ Fetching complete: 48 successful, 52 failed

[4/5] Saving fetched data...
✓ Saved JSON: nse_optio